In [5]:
import boto3
import json

aws_profile = 'default'

session = boto3.Session(profile_name=aws_profile)
client = session.client('kinesis')

client.list_streams()

{u'HasMoreStreams': False,
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '1060',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 03 Oct 2017 01:44:00 GMT',
   'server': 'Apache-Coyote/1.1',
   'x-amz-id-2': 'HlT2Ics0iICcUfIN2CjWGk/aqxHicwpkoGj3WWmUWPinSbcZ16zIuQwfC1st8mMK/bKlH07KfSdZWgg7KYlXXKCiBiwPxs2K',
   'x-amzn-requestid': 'e35b232a-f8bc-2325-b117-ce8ba68b4cf6'},
  'HTTPStatusCode': 200,
  'RequestId': 'e35b232a-f8bc-2325-b117-ce8ba68b4cf6',
  'RetryAttempts': 0},
 u'StreamNames': [u'AFLSurvey',
  u'Customer-Member-In-UAT',
  u'CustomerSales-Internal',
  u'CustomerSales-Internal-UAT',
  u'DataEnricher',
  u'DataEnricher-NZ',
  u'DataEnricher-UAT',
  u'Storm_LRL-au',
  u'TixTrack-CustomerSales-Internal-UAT',
  u'afl-attendance',
  u'afl-sales',
  u'aru-sales',
  u'attendance-in',
  u'attendance-in-nz',
  u'au-uat',
  u'backfill-au',
  u'backfill-internal',
  u'backfill-internal-mcg',
  u'backfill-nz',
  u'backfill-old',
  u'blackhole',
  u'blackhole-au',

In [12]:
# stream_name = 'storm-dev'
stream_name = 'referrals'
d_stream_desc = client.describe_stream(StreamName=stream_name)
print d_stream_desc

{'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'e873218d-f72b-408e-ba3f-cfb79765ea1c', 'HTTPHeaders': {'x-amzn-requestid': 'e873218d-f72b-408e-ba3f-cfb79765ea1c', 'content-length': '558', 'x-amz-id-2': 'qgeTZ83UziD5f9spU5UDSKdxj5AIEGDqkkPktw9Qus1PGr44uTJMKSqan+xHGTiTexF+lhEV6w0aJAA/bHWNc8y+p4FvoxUC', 'server': 'Apache-Coyote/1.1', 'date': 'Tue, 03 Oct 2017 01:46:34 GMT', 'content-type': 'application/x-amz-json-1.1'}}, u'StreamDescription': {u'HasMoreShards': False, u'RetentionPeriodHours': 24, u'StreamName': u'referrals', u'Shards': [{u'ShardId': u'shardId-000000000000', u'HashKeyRange': {u'EndingHashKey': u'340282366920938463463374607431768211455', u'StartingHashKey': u'0'}, u'SequenceNumberRange': {u'StartingSequenceNumber': u'49551061322225904759253682098696247552241686976212762626'}}], u'StreamARN': u'arn:aws:kinesis:ap-southeast-2:389920326251:stream/referrals', u'EnhancedMonitoring': [{u'ShardLevelMetrics': []}], u'StreamCreationTimestamp': datetime

In [13]:
# start_seq_no = '49569562892134134084378238761579934160273397074609831938'
start_seq_no = '49568548226781820946206637785869721311101902202802798594'
d_resp = client.get_shard_iterator(StreamName=stream_name,
                                   ShardId=d_stream_desc['StreamDescription']['Shards'][0]['ShardId'],
                                   ShardIteratorType='AT_SEQUENCE_NUMBER',
                                   StartingSequenceNumber=start_seq_no
                                  )
d_resp



In [13]:
d_resp = client.get_shard_iterator(StreamName=stream_name,
                                   ShardId=d_stream_desc['StreamDescription']['Shards'][0]['ShardId'],
                                   ShardIteratorType='LATEST'
                                  )
d_resp

ClientError: An error occurred (AccessDeniedException) when calling the GetShardIterator operation: User: arn:aws:iam::389920326251:user/deepakt is not authorized to perform: kinesis:GetShardIterator on resource: arn:aws:kinesis:ap-southeast-2:389920326251:stream/referrals

In [ ]:
d_resp = client.get_records(
    ShardIterator=d_resp['ShardIterator'],
    Limit=200
)

In [14]:
l_records = d_resp['Records']

len(l_records)

21

In [15]:
from StringIO import StringIO
import gzip
def decompress_data(compressed_data_blob):
    """
    Decompresses (unzips) given data blob
    :param compressed_data_blob:
    :return:
    """
    file_obj = StringIO(compressed_data_blob)
    try:
        with gzip.GzipFile(fileobj=file_obj, mode='rb') as gzipped_file:
            return gzipped_file.read()
    except IOError:
        # data blob is not in gzip format
        return compressed_data_blob

## Dibs record reader

In [16]:
import json
l_decompressed = []
for record in l_records:
#     print record
#     print '---------------------------'
    decompressed = decompress_data(record['Data'])
#     print 'decompressed: ' + decompressed
#     b64decoded = base64.b64decode(record['Data'])
#     print 'B64 decoded: ' + b64decoded
    l_decompressed.append(json.loads(decompressed))

In [11]:
l_decompressed

[{u'Count': 1,
  u'DateTime': u'2017-09-28T16:16:33.631+10:00',
  u'MSSQLRecordType': u'Sales.cdc.dbo_Payment_CT',
  u'Records': [{u'AcceptorId': None,
    u'Amount': 55760,
    u'AuthCode': u'1000022',
    u'AuthDate': None,
    u'AuthDateTime': u'2017-09-28T16:16:08.167+10:00',
    u'AuthReference': u'1000022',
    u'ClearCardNumber': None,
    u'CreatedBy': 9447,
    u'CreatedDate': u'2017-09-28T16:16:08.507+10:00',
    u'CustomerId': 14306629,
    u'Expiry': u'0320',
    u'HolderName': u'Test',
    u'IsCredit': False,
    u'MeansOfPayment': u'4775450000000014770143',
    u'MerchantId': u'ticketek',
    u'ModifiedDate': None,
    u'NewPaymentType': None,
    u'NumberMiddleSection': None,
    u'OriginalMeansOfPayment': None,
    u'PaymentId': 100068714,
    u'PaymentType': u'TVIS',
    u'Seller': u'WTROG1',
    u'TerminalNumber': None,
    u'Token': None,
    u'TrackingNumber': u'41ce0062-eaa0-496d-bb39-e288bbcaf6c7',
    u'tstamp': None}]},
 {u'Count': 1,
  u'DateTime': u'2017-09-28

In [17]:
import json
json.dumps(l_decompressed)

'[{"MSSQLRecordType": "Sales.cdc.dbo_Payment_CT", "Count": 1, "Records": [{"TerminalNumber": null, "NewPaymentType": null, "PaymentId": 100068713, "TrackingNumber": "57e85060-93ad-4e6d-b207-e1fdb76c86ec", "MeansOfPayment": "4374570000000015327021", "CustomerId": 14322784, "HolderName": "Test", "AuthDate": null, "IsCredit": false, "Expiry": "0320", "AuthDateTime": "2017-09-28T16:14:01.087+10:00", "MerchantId": "ticketek", "AuthReference": "1000021", "Amount": 27955, "NumberMiddleSection": null, "Seller": "WTROG1", "ClearCardNumber": null, "AuthCode": "1000021", "AcceptorId": null, "Token": null, "tstamp": null, "PaymentType": "TVIS", "ModifiedDate": null, "CreatedBy": 9447, "CreatedDate": "2017-09-28T16:14:01.460+10:00", "OriginalMeansOfPayment": null}], "DateTime": "2017-09-28T16:14:33.329+10:00"}, {"Count": 1, "Records": [{"h": -6097648294754985087, "ts": {"$timestamp": {"i": 2, "t": 1506579408}}, "o": {"orderId": "20170928,162", "customer": {"_id": 14306629, "postCode": "100000"}, "_